In [31]:
import paramiko
import os

In [32]:
def run_in_master(command):
    ssh = paramiko.SSHClient()
    ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())
    ssh.connect("namenode", username="root", password="pass")
    ssh_stdin, ssh_stdout, ssh_stderr = ssh.exec_command(f"cd /app/ && . /env_var_path.sh && {command}")
    return (ssh_stdout.readlines(), ssh_stderr.readlines())

def merge_results(path):
    run_in_master(f"hdfs dfs -cat {path}/part-* | hdfs dfs -put - {path}/merged.txt")
    
def get_data_from_output_path(path):
    return f"{path}/merged.txt"

def print_hdfs_output(path):
    raw = run_in_master(f"hdfs dfs -cat {get_data_from_output_path(path)}")[0]
    print("\n".join(raw[0:1000]))

In [33]:
os.system("cat /data/master_volume/pig_scripts/test.pig")

lines = LOAD '/datasets/covid-dataset.jsonl'  USING JsonLoader('date:chararray, location:chararray, total_cases:chararray, new_cases:chararray, total_deaths:chararray, new_deaths:chararray, new_cases_per_million:chararray');

data = foreach lines 
       GENERATE date, 
                location,
                total_cases,
                new_cases, 
                total_deaths, 
                new_deaths, 
                (new_cases_per_million == '' ? 0 : (int)new_cases_per_million) AS new_cases_per_million;

grouped = group data all;
avg = foreach grouped generate AVG(data.new_cases_per_million);
res = foreach data generate date, location, total_cases, new_cases, total_deaths, new_deaths, new_cases_per_million, avg.$0 AS average_new_cases_per_million;
STORE res INTO '/testoutput' using JsonStorage();

0

In [34]:
run_in_master("pig -x mapreduce /data/master_volume/pig_scripts/test.pig")

([],
 ['2023-05-20 17:22:36,518 INFO pig.ExecTypeProvider: Trying ExecType : LOCAL\n',
  '2023-05-20 17:22:36,519 INFO pig.ExecTypeProvider: Trying ExecType : MAPREDUCE\n',
  '2023-05-20 17:22:36,519 INFO pig.ExecTypeProvider: Picked MAPREDUCE as the ExecType\n',
  '2023-05-20 17:22:36,542 [main] INFO  org.apache.pig.Main - Apache Pig version 0.17.0 (r1797386) compiled Jun 02 2017, 15:41:58\n',
  '2023-05-20 17:22:36,542 [main] INFO  org.apache.pig.Main - Logging error messages to: /app/pig_1684603356538.log\n',
  '2023-05-20 17:22:36,690 [main] INFO  org.apache.pig.impl.util.Utils - Default bootup file /root/.pigbootup not found\n',
  '2023-05-20 17:22:36,714 [main] INFO  org.apache.hadoop.conf.Configuration.deprecation - mapred.job.tracker is deprecated. Instead, use mapreduce.jobtracker.address\n',
  '2023-05-20 17:22:36,714 [main] INFO  org.apache.pig.backend.hadoop.executionengine.HExecutionEngine - Connecting to hadoop file system at: hdfs://namenode:9000\n',
  '2023-05-20 17:22:

In [35]:
merge_results("/testoutput")

In [36]:
print_hdfs_output("/testoutput")

{"date":"NIU","location":"Oceania","total_cases":"Niue","new_cases":"2022-05-05","total_deaths":"9.0","new_deaths":"0.0","new_cases_per_million":0,"average_new_cases_per_million":10632.921378213763}

{"date":"NIU","location":"Oceania","total_cases":"Niue","new_cases":"2022-05-06","total_deaths":"9.0","new_deaths":"0.0","new_cases_per_million":0,"average_new_cases_per_million":10632.921378213763}

{"date":"NIU","location":"Oceania","total_cases":"Niue","new_cases":"2022-05-07","total_deaths":"9.0","new_deaths":"0.0","new_cases_per_million":0,"average_new_cases_per_million":10632.921378213763}

{"date":"NIU","location":"Oceania","total_cases":"Niue","new_cases":"2022-05-08","total_deaths":"9.0","new_deaths":"0.0","new_cases_per_million":0,"average_new_cases_per_million":10632.921378213763}

{"date":"NIU","location":"Oceania","total_cases":"Niue","new_cases":"2022-05-09","total_deaths":"9.0","new_deaths":"0.0","new_cases_per_million":0,"average_new_cases_per_million":10632.921378213763}

